In [ ]:
import pandas as pd


dfNF = pd.read_csv('daily_average.csv')
dfWF = pd.read_csv('daily_averageNoFire.csv')

dfNFSD = pd.read_csv('NaturalFiresTOPOGRAPHY.csv')
dfWFSD = pd.read_csv('LocationsWFire.csv')

In [ ]:
print(dfNF.columns)
print(dfWF.columns)

temp = 28_100
hum = 7_28

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

columns_to_drop = ['day', 'index', 'Unnamed: 0', 'index.1']

# Drop the columns
df = dfNF.drop(columns=columns_to_drop)

correlation_matrix = df.corr()

plt.figure(figsize=(30, 26))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')


plt.show()




In [ ]:
columns_to_keep = ['location', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.precipitation', 'hourly.wind_speed_10m', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.direct_normal_irradiance']
df = dfNF[columns_to_keep]
dfW = dfWF[columns_to_keep]
print(df.columns)
print(dfW.columns)

In [ ]:
columns_to_keep = ['location', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.precipitation', 'hourly.wind_speed_10m', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.direct_normal_irradiance']
df = dfNF[columns_to_keep]
dfW = dfWF[columns_to_keep]

df['wildfire'] = 1

df.set_index('location', inplace=True)

columns_days = {
    'hourly.temperature_2m': 8,
    'hourly.relative_humidity_2m': 8,
    'hourly.precipitation': 16,
    'hourly.wind_speed_10m': 16,
    'hourly.soil_temperature_28_to_100cm': 8,
    'hourly.soil_moisture_7_to_28cm': 8,
    'hourly.direct_normal_irradiance': 8
}

for column, days in columns_days.items():
    df = df.sort_index(ascending=False)
    df[f'{column}_lag'] = df.groupby(level=0)[column].shift(days)
    df = df.sort_index()

df.drop(columns=columns_days.keys(), inplace=True)

df.dropna(inplace=True)

# Without fire

dfW['wildfire'] = 0
dfW.set_index('location', inplace=True)

for column, days in columns_days.items():
    dfW = dfW.sort_index(ascending=False)
    dfW[f'{column}_lag'] = dfW.groupby(level=0)[column].shift(days)
    dfW = dfW.sort_index()

dfW.drop(columns=columns_days.keys(), inplace=True)
dfW.dropna(inplace=True)

print(df.columns)
print(dfW.columns)

dfNFSD['location'] = range(1, len(dfNFSD) + 1)
dfWFSD['location'] = range(1, len(dfWFSD) + 1)

# Convert 'CLC_CODE' to integer in dfNFSD and dfWFSD
dfNFSD['CLC_CODE'] = dfNFSD['CLC_CODE'].astype(int)
dfWFSD['CLC_CODE'] = dfWFSD['CLC_CODE'].astype(int)

print(dfNFSD.head())
print(dfWFSD.head())

#print(df.head())
#print(dfW.head())


variables_to_add = ['roughness', 'aspect', 'slope', 'CLC_CODE']

# For df
df = df.merge(dfNFSD[variables_to_add], left_index=True, right_on='location', how='left')

# For dfW
dfW = dfW.merge(dfWFSD[variables_to_add], left_index=True, right_on='location', how='left')

In [74]:
columns_to_keep = ['location', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.precipitation', 'hourly.wind_speed_10m', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.direct_normal_irradiance']
variables_to_add = ['roughness', 'aspect', 'slope', 'CLC_CODE']

df = dfNF[columns_to_keep]
dfW = dfWF[columns_to_keep]

df['wildfire'] = 1
dfW['wildfire'] = 0

df.set_index('location', inplace=True)
dfW.set_index('location', inplace=True)

columns_days = {
    'hourly.temperature_2m': 8,
    'hourly.relative_humidity_2m': 8,
    'hourly.precipitation': 16,
    'hourly.wind_speed_10m': 16,
    'hourly.soil_temperature_28_to_100cm': 8,
    'hourly.soil_moisture_7_to_28cm': 8,
    'hourly.direct_normal_irradiance': 8
}

location_features = {}

for location in df.index.unique():
    location_features[location] = {}

    for column, days in columns_days.items():
        df.sort_index(ascending=True, inplace=True)
        last_days = df.loc[location][-days:]
        location_features[location][f'{column}_lag'] = last_days[column].values

    location_features[location]['wildfire'] = df.loc[location]['wildfire'].values[0]

for location in dfNFSD['location']:
    for variable in variables_to_add:
        location_features[location][variable] = dfNFSD.loc[dfNFSD['location'] == location, variable].values[0]

for location in dfW.index.unique():
    location_features[int(location)+1770] = {}

    for column, days in columns_days.items():
        dfW.sort_index(ascending=True, inplace=True)
        last_days = dfW.loc[location][-days:]
        location_features[int(location)+1770][f'{column}_lag'] = last_days[column].values

    location_features[int(location)+1770]['wildfire'] = dfW.loc[location]['wildfire'].values[0]

for location in dfWFSD['location']:
    for variable in variables_to_add:
        location_features[int(location)+1770][variable] = dfWFSD.loc[dfNFSD['location'] == location, variable].values[0]

/tmp/ipykernel_5658/63745232.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wildfire'] = 1
/tmp/ipykernel_5658/63745232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfW['wildfire'] = 0


In [72]:
import json
import numpy as np

# Convert numpy arrays to lists and numpy int64 to int
for location in location_features:
    for feature in location_features[location]:
        if isinstance(location_features[location][feature], np.ndarray):
            location_features[location][feature] = location_features[location][feature].tolist()
        elif isinstance(location_features[location][feature], np.int64):
            location_features[location][feature] = int(location_features[location][feature])

# Save the dictionary into a JSON file
with open('location_features.json', 'w') as f:
    json.dump(location_features, f)

In [75]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

dfx = pd.DataFrame(location_features).T

# Split the DataFrame into features (X) and target (y)
X = dfx.drop('wildfire', axis=1)
y = dfx['wildfire']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = clf.predict(X_test)

# Evaluate the model
print('Accuracy:', accuracy_score(y_test, y_pred))

ValueError: setting an array element with a sequence.

In [76]:
import pandas as pd

# Initialize an empty DataFrame
dfx = pd.DataFrame()

# Loop over each location
for location, features in location_features.items():
    # Flatten the arrays and create new features
    for i, temp in enumerate(features['hourly.temperature_2m_lag']):
        dfx.loc[location, f'temperature_{i}'] = temp

    for i, humidity in enumerate(features['hourly.relative_humidity_2m_lag']):
        dfx.loc[location, f'humidity_{i}'] = humidity

    for i, precipitation in enumerate(features['hourly.precipitation_lag']):
        dfx.loc[location, f'precipitation_{i}'] = precipitation

    for i, wind_speed in enumerate(features['hourly.wind_speed_10m_lag']):
        dfx.loc[location, f'wind_speed_{i}'] = wind_speed

    for i, soil_temperature in enumerate(features['hourly.soil_temperature_28_to_100cm_lag']):
        dfx.loc[location, f'soil_temperature{i}'] = soil_temperature

    for i, soil_moisture in enumerate(features['hourly.soil_moisture_7_to_28cm_lag']):
        dfx.loc[location, f'soil_temperature{i}'] = soil_moisture

    for i, irradiance in enumerate(features['hourly.direct_normal_irradiance_lag']):
        dfx.loc[location, f'direct_normal_irradiance{i}'] = irradiance

    # Add the target variable
    dfx.loc[location, 'wildfire'] = features['wildfire']

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split the data into features and target variable
X = dfx.drop('wildfire', axis=1)
y = dfx['wildfire']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = clf.predict(X_test)

# Print the accuracy of the classifier
print('Accuracy:', accuracy_score(y_test, y_pred))

Accuracy: 0.8248587570621468


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)